This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/trajectories.html).  I recommend having both windows open, side-by-side!

In [ ]:
import numpy as np
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder,
                         KinematicTrajectoryOptimization, MeshcatVisualizer,
                         MeshcatVisualizerParams, MinimumDistanceConstraint,
                         Parser, PositionConstraint, RigidTransform, Solve,
                         Sphere, StartMeshcat)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import PublishPositionTrajectory
from manipulation.scenarios import AddPlanarIiwa, AddShape, AddWsg
from manipulation.utils import FindResource


In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

# Kinematic trajectory optimizaton

In [ ]:
def trajopt_shelves_demo():
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    iiwa = AddPlanarIiwa(plant)
    wsg = AddWsg(plant, iiwa, roll=0.0, welded=True)
    start = AddShape(plant, Sphere(0.02), "start")
    X_WStart = RigidTransform([0.8, 0, 0.65])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("start"), X_WStart)

    goal = AddShape(plant, Sphere(0.02), "goal")
    X_WGoal = RigidTransform([0.8, 0, 0.4])
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("goal"), X_WGoal)

    parser = Parser(plant)
    bin = parser.AddModelFromFile(FindResource("models/shelves.sdf"))
    plant.WeldFrames(plant.world_frame(),
                     plant.GetFrameByName("shelves_body", bin),
                     RigidTransform([0.9, 0, 0.4]))

    plant.Finalize()

    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat,
        MeshcatVisualizerParams(delete_prefix_initialization_event=False))

    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(context)

    q0 = plant.GetPositions(plant_context)
    gripper_frame = plant.GetFrameByName("body", wsg)
    start_frame = plant.GetFrameByName("start", start)
    goal_frame = plant.GetFrameByName("goal", goal)

    trajopt = KinematicTrajectoryOptimization(plant.num_positions(), 10)
    trajopt.AddDurationCost(1.0)
    trajopt.AddPositionBounds(plant.GetPositionLowerLimits(),
                              plant.GetPositionUpperLimits())
    trajopt.AddVelocityBounds(plant.GetVelocityLowerLimits(),
                              plant.GetVelocityUpperLimits())
    trajopt.AddNominalPositionCost(q0)

    # TODO(russt): Remove this once the solution is otherwise constrained
    trajopt.AddDurationBounds(.1, 20)
    trajopt.AddEqualTimeIntervalConstraints()

    # start constraint
    start_constraint = PositionConstraint(plant, start_frame, [0, 0, 0],
                                          [0, 0, 0], gripper_frame, [0, 0.1, 0],
                                          plant_context)
    trajopt.AddPathConstraint(start_constraint, 0)

    # goal constraint
    goal_constraint = PositionConstraint(plant, goal_frame, [0, 0, 0],
                                         [0, 0, 0], gripper_frame, [0, 0.1, 0],
                                         plant_context)
    trajopt.AddPathConstraint(goal_constraint, 1)

    # start and end with zero velocity
    trajopt.AddFixedPathVelocityConstraint(np.zeros((3,1)), 0)
    trajopt.AddFixedPathVelocityConstraint(np.zeros((3,1)), 1)

    # collision constraints
    collision_constraint = MinimumDistanceConstraint(plant, 0.001,
                                                     plant_context, None, 0.01)
    evaluate_at_s = np.linspace(0, 1, 25)
    for s in evaluate_at_s:
        trajopt.AddPathConstraint(collision_constraint, s)

    result = Solve(trajopt.prog())
    if not result.is_success():
        print("Trajectory optimization failed")
        print(result.get_solver_id().name())

    PublishPositionTrajectory(trajopt.ReconstructPath(result), context, plant,
                              visualizer)

trajopt_shelves_demo()
